In [1]:
import os
os.chdir("/RAFT-Stereo/RGBNIR") # Fix with forder root path

In [2]:
from net.image_fusion import HSVNet
import torch


model = HSVNet().cuda()
model.load_state_dict(torch.load("weights/model_image_fusion.pth")["model_state_dict"])
model.eval()



/RAFT-Stereo/RGBNIR/net/utils.py:5: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy import interpolate


HSVNet(
  (encoder): BasicEncoder(
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (relu1): ReLU(inplace=True)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [ ]:
import numpy as np
from visualize.vis_utils.widget import FrameExplorer
from visualize.vis_utils.inputs import get_valid_input_from_path
import matplotlib.pyplot as plt


def plot_dataset(path: str):
    inputs = get_valid_input_from_path(path)
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 4, 1)
    plt.imshow(inputs[0][0].permute(1, 2, 0).numpy().astype(np.uint8))
    plt.subplot(1, 4, 2)
    plt.imshow(inputs[2][0].permute(1, 2, 0).numpy().astype(np.uint8), cmap="gray")
    plt.subplot(1, 4, 3)
    plt.imshow(np.zeros((540, 720), np.uint8), cmap="gray")
    plt.scatter(
        inputs[4][..., 0],
        inputs[4][..., 1],
        c=inputs[4][..., 2],
        s=1,
        cmap="magma",
        vmin=0,
        vmax=32,
    )
    plt.xlim(0, 720)
    with torch.no_grad():
        fusion = model(
            [inputs[0].cuda(), inputs[1].cuda()], [inputs[2].cuda(), inputs[3].cuda()]
        )
    plt.subplot(1, 4, 4)
    plt.imshow(fusion.cpu()[0].permute(1, 2, 0).numpy().astype(np.uint8))
    plt.show()


FrameExplorer(plot_dataset)